# **Business Objective:** 
The difference between the use of Cyclistic bikes between Casual riders and annual members in order to increase the company revenue by increasing annual members.


# **Description of Data Source:**
The data is available and regularly updated in the zip format file on the website <Index of bucket "divvy-tripdata">. The data for the past 12 months from April 2023 to May 2020 was downloaded from this website and arranged in a separate folder in my Drive, also a copy of the original data is made in order to refer to later steps in the analysis if needed. The data is organized with the ride_id as the field name and with respect to this id different details of the ride such as start time, end time, starting and ending station of the trip, and also the member type also mentioned whether the member is an annual member or casual user. After filtering and viewing data it can be seen that starting or ending station names or latitude and longitude values of a few rides are missing. Since the data is provided from a trusted source and also the privacy of the user is maintained by not showing their sensitive credentials such as name, credit card number, etc. so the data integrity is maintained for initial level analysis.

# **Steps Taken for Cleaning and Manipulation of Data:**
* First of all the data is checked in Excel for any missing or wrong value within all the columns and it was found that a significant amount of values are missing in columns like start_station_name, end_station_name, start_lat, start long, end_lat, end_long. These data values were not omitted as they were making a significant amount of data and also these data points were not creating a great impact on our analysis.
* Then the days_of_week were found in Excel using the text function for each month. 
* After loading the necessary libraries, all these CSV files are uploaded into the R environment for further analysis, I chose Rstudio as it supports this much amount of large data.
* After that, all the data for 12 months are then aggregated into one table i.e., ‘all_trips’.




In [ ]:
library(tidyverse)
library(ggplot2)
library(lubridate)
install.packages("farver")
getwd()
#setwd("E:/Coursera Data Science Tasks/Google Capstone/Week 2/Edited data of cycle/Edited csv")
library(readr)
April_2023<- read_csv("April 2023.csv")
March_2023<- read_csv("March 2023.csv")
Feb_2023 <- read_csv("Feb 2023.csv")
Jan_2023 <- read_csv("Jan 2023.csv")
Dec_2022 <- read_csv("Dec 2022.csv")
Nov_2022 <- read_csv("Nov 2022.csv")
Oct_2022 <- read_csv("Oct 2022.csv")
Sept_2022 <- read_csv("Sept 2022.csv")
Aug_2022 <- read_csv("Aug 2022.csv")
Jul_2022 <- read_csv("Jul 2022.csv")
Jun_2022 <- read_csv("Jun 2022.csv")
May_2022 <- read_csv("May 2022.csv")
colnames(March_2023)
str(April_2023)
##str(March_2023)
##str(Feb_2023)
library(hms)
April_2023 <-mutate(April_2023,starting_time = as_hms(starting_time))
April_2023 <-mutate(April_2023,ending_time = as_hms(ending_time))
str(April_2023)
str(March_2023)

all_trips <- bind_rows(April_2023,March_2023)
str(April_2023)
all_trips <-bind_rows(all_trips,Feb_2023)
all_trips <- bind_rows(all_trips,Jan_2023,Dec_2022,Nov_2022,Oct_2022,Sept_2022,Jul_2022,Jun_2022,May_2022)
str(all_trips)

* The started_date formatted into mdy_hm format for further analysis
* Then with the help of ymd library months and years were found separately

In [ ]:
all_trips$starting_date <- mdy(all_trips$starting_date)
all_trips$months<- months(ymd(all_trips$starting_date))
str(all_trips)
all_trips$years<- year((ymd(all_trips$starting_date)))
all_trips$days<- day((ymd(all_trips$starting_date)))

* After correcting the format from string to mdy_hm of started_at and ended_at values ride_length was calculated
* The ride_length i.e., duration of ride was calculated by subtracting starting and ending time using difftime()  function.

In [ ]:
all_trips$started_at <- mdy_hm(all_trips$started_at)
all_trips$ended_at <- mdy_hm(all_trips$ended_at)
all_trips$ride_length <- as.numeric(difftime(all_trips$ended_at,all_trips$started_at, units = "secs"))

* Data points with negative ride_length are omitted as starting time cannot be less than the ending time, but before omitting a copy of data set was made as data is being removed.
* So now the new table all_trips_v2 is formed and again checked for any na values.


In [ ]:
str(all_trips)
is.numeric(all_trips$ride_length)
all_trips_v2 <- all_trips[!( all_trips$ride_length<=0),]
str(all_trips)
View(all_trips_v2)
sum(is.na(all_trips_v2$ride_length))

# **Performing Arithmetic Calculations:**
* First of all max, min, mean, and median of ride length were calculated. 
* Further, different correlation between both type of users and the number of rides on each day was found and viewed.
* Similarly correlation between both type of users and average_ride length each day was calculated and viewed.
* Preference of users for the type of bike they use more often was found and viewed.
* Overall yearly analysis is done by finding out the number of rides used by each user in different quarters of the year


# **Summary of Analysis:**
* The summary for max, min, mean, and median were found out as below:

In [ ]:
mean(all_trips_v2$ride_length)
median(all_trips_v2$ride_length)
max(all_trips_v2$ride_length)
min(all_trips_v2$ride_length)

summary(all_trips_v2$ride_length)

* The mean, max, min and median was calculated again with respect to the member type.

In [ ]:
aggregate(all_trips_v2$ride_length~all_trips_v2$member_casual,FUN = mean)
aggregate(all_trips_v2$ride_length~all_trips_v2$member_casual,FUN = median)
aggregate(all_trips_v2$ride_length~all_trips_v2$member_casual,FUN = max)
aggregate(all_trips_v2$ride_length~all_trips_v2$member_casual,FUN = min)

* Now calculating average ride duration for each day for both the customer.

In [ ]:
#checking average ride length for each day of members and casual users
aggregate(all_trips_v2$ride_length~all_trips_v2$member_casual+all_trips_v2$day_of_weeks, FUN=mean)

* Since the days were not sorted out so we then sorted it and again calculated the average ride duration.

In [ ]:
#Now sordering days of the week.
all_trips_v2$day_of_weeks <- ordered(all_trips_v2$day_of_weeks, levels = c('Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday'))

#Again checking the avg ride time of each day of both type of customers
aggregate(all_trips_v2$ride_length~all_trips_v2$member_casual+all_trips_v2$day_of_weeks, FUN = mean)

* Now calculating average ride length in more appropiate way.

In [ ]:
#checking average rides on every day for both type of customer
all_trips_v2 %>%
  group_by(member_casual,day_of_weeks)%>%
  summarise(number_of_rides = n(),average_duration = mean(ride_length))%>%
  arrange(member_casual, day_of_weeks)


* Now dividing the years in 4 Quarters for further analysis

In [ ]:
all_trips_v2$quaterly_dates <-as.numeric( quarter(all_trips_v2$ended_at))
View(all_trips_v2)
#dividing in quarters

first_quater <-  filter(all_trips_v2, quaterly_dates=="1")
#View(first_quater)
second_quater <-  filter(all_trips_v2, quaterly_dates=="2")
#View(second_quater)
third_quater <-  filter(all_trips_v2, quaterly_dates=="3")
#View(third_quater)
fourth_quater <-  filter(all_trips_v2, quaterly_dates=="4")
#View(fourth_quater)
#number of rides
count_of_1st_quater <- count(first_quater)
#View(count_of_1st_quater)
count_of_2nd_quater <- count(second_quater)
count_of_3rd_quater <- count(third_quater)
count_of_4th_quater <- count(fourth_quater)

* Now we will be visualizing data with different perspectives using ggplot2 commands.
* Firstly, we will be viewing number of rides for each type of user.

In [ ]:
#Let's visualize the number of rides everyday for both customers
all_trips_v2 %>%
  group_by(member_casual,day_of_weeks)%>%
  summarise(number_of_rides = n(),average_duration = mean(ride_length))%>%
  arrange(member_casual, day_of_weeks)%>%
  ggplot(aes(x=day_of_weeks, y = number_of_rides, fill = member_casual))+geom_col(position = "dodge")

* Now we will see the average rides of each customer on each day.

In [ ]:
#Now we will see graph of average ride on each day for both customers
all_trips_v2 %>%
  group_by(member_casual,day_of_weeks)%>%
  summarise(number_of_rides = n(),average_duration = mean(ride_length))%>%
  arrange(member_casual, day_of_weeks)%>%
  ggplot(aes(x=day_of_weeks, y = average_duration, fill = member_casual))+geom_col(position = "dodge")

* Finding out the number of rides of each customer on in different quarters of the year.

In [ ]:
#viewing number of rides of member and casual w.r.t types of member
all_trips_v2%>%
  group_by(member_casual, quaterly_dates)%>%
  summarise(number_of_rides=n(), member_casual)%>%
  arrange(member_casual, quaterly_dates)%>%
  ggplot(aes(x = quaterly_dates , y = number_of_rides , fill = member_casual ))+geom_col(position = "dodge")


* Also as we are having different bike types therefore now finding out the ride preferences of each user.

In [ ]:
#now viewing preferance of customers w.r.t bike type
  all_trips_v2%>%
    group_by(member_casual, rideable_type)%>%
    summarise(number_of_rides=n(), member_casual)%>%
    arrange(member_casual, rideable_type)%>%
    ggplot(aes(x = rideable_type , y = number_of_rides , fill = member_casual ))+geom_col(position = "dodge")
  

# **TOP THREE RECOMMENDATIONS:**
Based on the above findings my top 3 recommendations would be.

* Since casual users use bike more oftenly on weekends so we can run marketing campaigns by encouraging the use of bikes not only for leisure purpose on weekends but also for daily day commute by raising awareness about the environmental and health benefits when using cycle. Also, we can introduce a discount for the weekdays.

* As we can see that the average ride duration for casual users on each is interestingly more than member users, so we can attract these casual users by limiting their usage and they will eventually have to take the membership to ride for long duration
 
* In order to increase membership of casual members we can target them during in Q1 and Q4 as they use bikes almost half then the member user as these months are cold. We can have a discount for winter times and raise awareness of exercise in winter.


# **LIMITATIONS WITH DATA AND IT'S IMPACT:**

The limitations was of:
* Missing station names of starting and ending end, with the help of which we could have find the most prominent stations.
* The age and gender information missing, with which we could have targeted specific audience according to insights.